In [2]:
from torchvision import datasets
from torchvision import transforms

from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

import torchvision.models as models
import torch.nn as nn
import torch

In [3]:
IMAGE_SIZE = 224
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]
BATCH_SIZE = 16
NUM_WORKERS = 2
NUM_CLASSES = 9

test_transform = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
])

test_dataset_path = '../../../dataset/test'
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=test_transform)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

In [4]:
def test_model(model, loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [5]:
model = models.mobilenet_v3_large(weights=None)

for param in model.features.parameters():
    param.requires_grad = False

old_linear = model.classifier[3]
in_feats = old_linear.in_features
classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_feats, NUM_CLASSES)
)
model.classifier[3] = classifier

trained_model_path = "./mobilenetv3-05202025213410.pth"
model.load_state_dict(torch.load(trained_model_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

criterion = torch.nn.CrossEntropyLoss()

In [6]:

test_loss, test_accuracy = test_model(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.2115, Test Accuracy: 0.9276


In [7]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n",
      classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Confusion Matrix:
 [[162   0   2   0   0   6   0   0   0]
 [  0 121  11   0   0   0   0   2   0]
 [  6  15 142   0   0   0   0   0   0]
 [  1   0   1 145   1   0   0   2   0]
 [  0   0   1   8 138   0   0   3   0]
 [  3   0   1   0   0  19   0   0   1]
 [  0   0   0   0   0   0  21   0   0]
 [  2   5   0   1   0   0   0 164   0]
 [  0   0   0   0   0   0   0   0  11]]

Classification Report:
                      precision    recall  f1-score   support

         algal_spot       0.93      0.95      0.94       170
       brown_blight       0.86      0.90      0.88       134
        gray_blight       0.90      0.87      0.88       163
            healthy       0.94      0.97      0.95       150
         helopeltis       0.99      0.92      0.96       150
           red-rust       0.76      0.79      0.78        24
red-spider-infested       1.00      1.00      1.00        21
           red_spot       0.96      0.95      0.96       172
         white-spot       0.92      1.00      0.96    